In [13]:
import os
from selenium import webdriver
from selenium.webdriver.common.by import By
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
from selenium.webdriver.common.keys import Keys
from selenium.common.exceptions import StaleElementReferenceException
import urllib.parse
import time
import re
from bs4 import BeautifulSoup
import pandas as pd

In [14]:
os.environ['PATH'] += r'C:\Users\gavvi\ChromeDrivers\chrome-win64\chrome-win64'

In [15]:
driver = webdriver.Chrome()
url = "https://skb-insilico.com/dlip/compound-search/ppi-library/rule-based"
driver.get(url)

driver.implicitly_wait(5)

In [16]:
# Click on the "All" button
all_btn = driver.find_element(By.XPATH, "//button[text()=' All']")
all_btn.click()

driver.implicitly_wait(2)

In [17]:
search_btn = WebDriverWait(driver, 10).until(
    EC.element_to_be_clickable((By.CLASS_NAME, 'btn-green'))
)
search_btn.click()

In [18]:
table = WebDriverWait(driver, 20).until(
            EC.presence_of_element_located((By.CLASS_NAME, "dataTables_scrollBody"))
        )

# Extract the HTML content of the table
table_html = table.get_attribute('outerHTML')

# Use BeautifulSoup to parse the HTML
soup = BeautifulSoup(table_html, 'html.parser')

# Extract column headers
header = [th.text for th in soup.find_all('th')]

# Construct empty DataFrame
df = pd.DataFrame([], columns=header)

In [19]:
df

,DLiP-ID,Mol Image,MW,XLogP,HBA,HBD,PSA,nRotatableBonds,nRings


In [20]:
def update_dataframe_on_new_page(driver, base_url, existing_df):
    # Extract the HTML content of the table
    table_html = driver.find_element(By.CLASS_NAME, "dataTables_scrollBody").get_attribute('outerHTML')

    # Use BeautifulSoup to parse the HTML
    soup = BeautifulSoup(table_html, 'html.parser')

    data = []

    # Extract table data manually
    for row in soup.find_all('tr')[1:]:
        row_data = [td.text for td in row.find_all('td')]

        # Extract the DLiP-ID and Canonical SMILES(RDKit) links
        dlip_id_link = row.find('a', {'href': re.compile(r'/dlip/compound/')})
        smiles_link = row.find('a', {'href': re.compile(r'/dlip/compound/[A-Z]\d+')}), 

        # Navigate to the DLiP-ID link
        dlip_id_url = urllib.parse.urljoin(base_url, dlip_id_link['href'])
        driver.get(dlip_id_url)

        # Extract the Canonical SMILES(RDKit) value
        smiles_value = driver.find_element(By.XPATH, '//td[text()="Canonical SMILES(RDKit)"]/following-sibling::td').text

        # Replace the Mol Image value with the Canonical SMILES(RDKit)
        mol_image_index = existing_df.columns.get_loc("Mol Image")
        row_data[mol_image_index] = smiles_value

        # Return to the initial page
        driver.back()

        # Append the modified row_data to the DataFrame
        data.append(row_data)

    # Ensure the columns are in the correct order
    new_df = pd.DataFrame(data, columns=existing_df.columns)

    # Concatenate DataFrames
    updated_df = pd.concat([existing_df, new_df], ignore_index=True)

    return updated_df


In [21]:
page_threshold = 609

In [24]:
base_url = "https://skb-insilico.com"

# Updated the empty data frame with the content of the first page using our helper function
df = update_dataframe_on_new_page(driver, base_url, df)

# Loop through pages until the last page
page_number = 1
while True:
    try:
        # Wait for the loading overlay to disappear
        WebDriverWait(driver, 20).until(
            EC.invisibility_of_element_located((By.CLASS_NAME, "loadingoverlay"))
        )
        
        # Find the "Next" button
        next_button = driver.find_element(By.XPATH, '//*[@id="compound-list-table_next"]/a')
        
        # Click on the "Next" button
        next_button.click()

        # Wait for the table to be present on the next page
        time.sleep(3) 
        table = WebDriverWait(driver, 20).until(
            EC.presence_of_element_located((By.CLASS_NAME, "dataTables_scrollBody"))
        )

        # Update the old dataframe with the content of the next website page using our helper function
        df = update_dataframe_on_new_page(driver, base_url, df)

        page_number += 1

        # Check if there are more pages
        if 'disabled' in next_button.get_attribute('class') or page_number > page_threshold:
            break

    except StaleElementReferenceException:
        continue


NoSuchElementException: Message: no such element: No node with given id found
  (Session info: chrome=119.0.6045.160); For documentation on this error, please visit: https://www.selenium.dev/documentation/webdriver/troubleshooting/errors#no-such-element-exception
Stacktrace:
	GetHandleVerifier [0x00007FF7C2F582B2+55298]
	(No symbol) [0x00007FF7C2EC5E02]
	(No symbol) [0x00007FF7C2D805AB]
	(No symbol) [0x00007FF7C2D6EF89]
	(No symbol) [0x00007FF7C2D6D041]
	(No symbol) [0x00007FF7C2D6D9CE]
	(No symbol) [0x00007FF7C2D6D900]
	(No symbol) [0x00007FF7C2D8E706]
	(No symbol) [0x00007FF7C2D859E9]
	(No symbol) [0x00007FF7C2D85AF3]
	(No symbol) [0x00007FF7C2D84298]
	(No symbol) [0x00007FF7C2D8732A]
	(No symbol) [0x00007FF7C2DFB12B]
	(No symbol) [0x00007FF7C2DE20AA]
	(No symbol) [0x00007FF7C2DFAAA4]
	(No symbol) [0x00007FF7C2DE1E83]
	(No symbol) [0x00007FF7C2DB670A]
	(No symbol) [0x00007FF7C2DB7964]
	GetHandleVerifier [0x00007FF7C32D0AAB+3694587]
	GetHandleVerifier [0x00007FF7C332728E+4048862]
	Get

In [43]:
# Save the DataFrame to a CSV file
csv_file_path = "temp_failed_.csv"
df.to_csv(csv_file_path, index=False)

In [38]:
df_no_duplicates = df.drop_duplicates()


In [39]:
df_no_duplicates

,DLiP-ID,Mol Image,MW,XLogP,HBA,HBD,PSA,nRotatableBonds,nRings
0,D00000,COc1cccc2c1OCC21CCN(C(=O)CC2(c3cccc(Br)c3)CCNC...,499.449,3.49,4,1,50.8,4,5
1,D00001,COc1ccccc1Cn1nc(C)c(C(=O)N2CCC(CN3Cc4ccc(F)cc4...,490.579,4.05,5,0,67.67,6,5
2,D00002,COc1cccc(C2(CC(=O)NC3(Cc4ccc(Cl)cc4)CCS(=O)(=O...,505.08,3.868,5,2,84.5,7,4
3,D00003,NC1CCN(Cc2ccccc2C(=O)NC2CCN(Cc3ccccc3C(=O)O)CC...,450.583,2.666,5,3,98.9,7,4
4,D00004,COc1cccc(C2(CC(=O)Nc3ccccc3N3CCC(C(=O)O)CC3)CC...,451.567,2.902,5,3,90.9,7,4
...,...,...,...,...,...,...,...,...,...
15259,D00BQH,CCOc1cccc(C(=O)N[C@H]2CC[C@H](NCc3cccc(C(=O)NC...,598.748,4.2,7,3,125.55,11,5
15260,D00BQI,COCCOc1cccnc1C(=O)N[C@H]1CC[C@H](NCc2cccc(C(=O...,643.76,3.907,7,2,113.1,11,6
15261,D00BQJ,O=C(Nc1ccccc1Cl)NC1CCN(C(=O)c2cccc(CN[C@H]3CC[...,620.238,6.435,4,4,102.57,8,6
15262,D00BQK,CCOc1cccc(C(=O)N[C@H]2CC[C@H](NCc3cccc(C(=O)N4...,625.77,5.002,7,2,113.1,10,6


In [163]:
# Save the DataFrame to a CSV file
csv_file_path = "temp1.csv"
df.to_csv(csv_file_path, index=False)

In [34]:
df_no_duplicates = df.iloc[:, 1:].drop_duplicates()

In [40]:
# Assuming "DLiP_AD.csv" is in your current directory
file_path = "DLiP_AD.csv"

# Read the CSV file into a DataFrame
df_late = pd.read_csv(file_path)

In [41]:
df_late

,DLiP-ID,Mol Image,MW,XLogP,HBA,HBD,PSA,nRotatableBonds,nRings
0,D00000,NaN,499.449,3.490,4,1,50.80,4,5
1,D00001,NaN,490.579,4.050,5,0,67.67,6,5
2,D00002,NaN,505.080,3.868,5,2,84.50,7,4
3,D00003,NaN,450.583,2.666,5,3,98.90,7,4
4,D00004,NaN,451.567,2.902,5,3,90.90,7,4
...,...,...,...,...,...,...,...,...,...
15223,D00BQH,NaN,598.748,4.200,7,3,125.55,11,5
15224,D00BQI,NaN,643.760,3.907,7,2,113.10,11,6
15225,D00BQJ,NaN,620.238,6.435,4,4,102.57,8,6
15226,D00BQK,NaN,625.770,5.002,7,2,113.10,10,6


In [42]:
df_no_duplicates_ = df_late.drop_duplicates()
df_no_duplicates_

,DLiP-ID,Mol Image,MW,XLogP,HBA,HBD,PSA,nRotatableBonds,nRings
0,D00000,NaN,499.449,3.490,4,1,50.80,4,5
1,D00001,NaN,490.579,4.050,5,0,67.67,6,5
2,D00002,NaN,505.080,3.868,5,2,84.50,7,4
3,D00003,NaN,450.583,2.666,5,3,98.90,7,4
4,D00004,NaN,451.567,2.902,5,3,90.90,7,4
...,...,...,...,...,...,...,...,...,...
15209,D00BQH,NaN,598.748,4.200,7,3,125.55,11,5
15210,D00BQI,NaN,643.760,3.907,7,2,113.10,11,6
15211,D00BQJ,NaN,620.238,6.435,4,4,102.57,8,6
15212,D00BQK,NaN,625.770,5.002,7,2,113.10,10,6
